In [5]:
# map with all CZ regions
from bokeh.models.widgets import Panel, Tabs 
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure, ColumnDataSource
from bokeh.tile_providers import get_provider, Vendors
from bokeh.palettes import PRGn, RdYlGn, PuRd, Reds, Spectral6, RdGy
from bokeh.transform import linear_cmap,factor_cmap
from bokeh.layouts import row, column
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter, Circle, HoverTool, LabelSet, Paragraph, ColumnDataSource, DataTable, DateFormatter, TableColumn
from bokeh.layouts import layout, column
from bokeh.models.widgets import (Div)

import numpy as np
import pandas as pd
import datetime
from functools import reduce
import seaborn as sns

# ---------------- okresy data preparation + coordinates transforamtion (to mercator)

def x_coord(x, y):              # transformation - geopraphical coord. to mercator
 lat = x
 lon = y
    
 r_major = 6378137.000
 x = r_major * np.radians(lon)
 scale = x/lon
 y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 +
     lat * (np.pi/180.0)/2.0)) * scale
 return (x, y)

df_input={"okres":["Bruntál","Frýdek-Místek","Karviná","Nový Jičín","Opava","Ostrava-Město",
                  "Jeseník","Olomouc","Prostějov","Přerov","Šumperk",
                  "Kroměříž","Uherské Hradiště","Vsetín","Zlín",
                  "Praha",
                  "Blansko", "Brno-město","Brno-venkov","Břeclav","Hodonín","Vyškov","Znojmo",
                  "Havlíčkův Brod","Jihlava","Pelhřimov","Třebíč","Žďár nad Sázavou",
                  "Chrudim","Pardubice","Svitavy","Ústí nad Orlicí",
                  "Hradec Králové","Jičín","Náchod","Rychnov nad Kněžnou","Trutnov",
                  "Česká Lípa","Jablonec nad Nisou","Liberec","Semily",
                  "Benešov", "Beroun","Kladno","Kolín","Kutná Hora","Mělník","Mladá Boleslav","Nymburk", "Praha-východ","Praha-západ","Příbram","Rakovník",
                  "České Budějovice","Český Krumlov","Jindřichův Hradec","Písek","Prachatice","Strakonice","Tábor",
                  "Cheb","Karlovy Vary","Sokolov",
                  "Domažlice","Klatovy","Plzeň-město","Plzeň-jih","Plzeň-sever","Rokycany","Tachov",
                  "Děčín","Chomutov","Litoměřice","Louny","Most","Teplice","Ústí nad Labem"], 
          "kraj_NUTS3":["CZ080","CZ080","CZ080","CZ080","CZ080","CZ080",
                       "CZ071","CZ071","CZ071","CZ071","CZ071",
                       "CZ072","CZ072","CZ072","CZ072",
                       "CZ010",
                       "CZ064","CZ064","CZ064","CZ064","CZ064","CZ064", "CZ064",
                       "CZ063","CZ063","CZ063","CZ063","CZ063",
                       "CZ053","CZ053","CZ053","CZ053",
                       "CZ052","CZ052","CZ052","CZ052","CZ052",
                       "CZ051","CZ051","CZ051","CZ051",
                       "CZ020","CZ020","CZ020","CZ020","CZ020","CZ020", "CZ020","CZ020","CZ020","CZ020","CZ020","CZ020",
                       "CZ031","CZ031","CZ031","CZ031","CZ031","CZ031", "CZ031",
                       "CZ041","CZ041","CZ041",
                       "CZ032","CZ032","CZ032","CZ032","CZ032","CZ032","CZ032",
                       "CZ042","CZ042","CZ042","CZ042","CZ042","CZ042","CZ042"],
          "okres_LAU1":["CZ0801","CZ0802","CZ0803","CZ0804","CZ0805","CZ0806",
                       "CZ0711","CZ0712","CZ0713","CZ0714","CZ0715",
                       "CZ0721","CZ0722","CZ0723","CZ0724",
                       "CZ0100",
                       "CZ0641","CZ0642","CZ0643","CZ0644","CZ0645","CZ0646", "CZ0647",
                       "CZ0631","CZ0632","CZ0633","CZ0634","CZ0635",
                       "CZ0531","CZ0532","CZ0533","CZ0534",
                       "CZ0521","CZ0522","CZ0523","CZ0524","CZ0525",
                       "CZ0511","CZ0512","CZ0513","CZ0514",
                       "CZ0201","CZ0202","CZ0203","CZ0204","CZ0205","CZ0206", "CZ0207","CZ0208","CZ0209","CZ020A","CZ020B","CZ020C",
                       "CZ0311","CZ0312","CZ0313","CZ0314","CZ0315","CZ0316", "CZ0317",
                       "CZ0411","CZ0412","CZ0413",
                       "CZ0321","CZ0322","CZ0323","CZ0324","CZ0325","CZ0326","CZ0327",
                       "CZ0421","CZ0422","CZ0423","CZ0424","CZ0425","CZ0426","CZ0427"],
          "latitude": [49.988232, 49.682561, 49.861098, 49.596599, 49.940066, 49.820923,
                      50.223847, 49.590463, 49.472283, 49.451107, 49.964742,
                      49.294681, 49.067078, 49.339914, 49.229004,
                      50.082062,
                      49.365749, 49.191729, 49.035858, 48.748581, 48.861839, 49.282222, 48.862443,
                      49.602843, 49.403276, 49.428509, 49.226549, 49.574084,
                      49.954686, 50.030921, 49.759101, 49.972611,
                      50.207966, 50.434692, 50.414404, 50.164632, 50.570516,
                      50.681658, 50.730967, 50.761099, 50.610618,
                      49.789151, 49.970051, 50.145457, 50.027388, 49.953627, 50.365000, 50.419563, 50.180723, 50.157412, 50.060318, 49.680894, 50.105440,
                      48.986825, 48.818309, 49.150045, 49.313065, 49.008237, 49.263026, 49.414851,
                      50.078204, 50.236768, 50.182698,
                      49.446549, 49.398864, 49.749709, 49.586797, 49.927326, 49.744869, 49.802828,
                      50.780989, 50.469315, 50.540443, 50.360890, 50.510325, 50.648221, 50.664377],
          "longitude": [17.463576, 18.358651, 18.543283, 18.015774, 17.894213, 18.262524,
                       17.200008, 17.254502, 17.111340, 17.447462, 16.974617,
                       17.393367, 17.481180, 17.988480, 17.673415,
                       14.416865,
                       16.649480, 16.608515, 16.551418, 16.876099, 17.114064, 16.997414, 16.037517,
                       15.579578, 15.589201, 15.221065, 15.872449, 15.935233,
                       15.790516, 15.772932, 16.472454, 16.409975,
                       15.835992, 15.370135, 16.178177, 16.283982, 15.912695,
                       14.548927, 15.171210, 15.060655, 15.327588,
                       14.687265, 14.076226, 14.114346, 15.195141, 15.280175, 14.478942, 14.909244, 15.039684, 14.779787, 14.175801, 13.996839, 13.722801,
                       14.475441, 14.316822, 15.006456, 14.152659, 13.996990, 13.906124, 14.679119,
                       12.373971, 12.863628, 12.654435,
                       12.922534, 13.285901, 13.391699, 13.414021, 13.297464, 13.585913, 12.642058,
                       14.220062, 13.402954, 14.127376, 13.803263, 13.631646, 13.835729, 14.051109],
          "population":[91597,214660,246324,151577,176236,320145,
                       37968, 235472, 108646, 129512, 120417,
                       105343, 142226, 143334, 191652,
                       1324277,
                       109136, 381346, 224642, 116291, 153943, 92280, 114351,
                       94915, 113628, 72302, 110810, 118158,
                       104613, 175441, 104333, 138275,
                       164283, 80045, 109958, 79383, 117978,
                       103300, 90667, 175626, 74097,
                       99414, 95058, 166483, 102623, 75828, 109302, 130365, 100886, 185178, 149338, 115104, 55562,
                       195903, 61556, 90692, 71587, 50978, 70772, 102595,
                       91634, 114818, 88212,
                       62062, 86405, 194280, 63488, 79979, 49349, 54336,
                       129542, 124946, 119668, 86691, 111708, 129072, 119338]}  # DS from dictionary
df_okresy = pd.DataFrame(df_input)
df_okresy.index += 1               # start index with 1 not with zero 

df_okresy['coordinates'] = list(zip(df_okresy['latitude'], df_okresy['longitude']))  # create new column with lat and long as tuple (together)
mercators = [x_coord(x, y) for x, y in df_okresy['coordinates'] ]   # Obtain list of mercator coordinates  

df_okresy['mercator'] = mercators       # add coordinates to the mercator column

df_okresy[['mercator_x', 'mercator_y']] = df_okresy['mercator'].apply(pd.Series) # split into two separate columns

# print (df_okresy.head(15))

# ------------------------------------- Covid data preparation

df_MZ = pd.read_csv ("https://onemocneni-aktualne.mzcr.cz/api/v2/covid-19/osoby.csv")
df_MZ2 = pd.read_csv ("https://onemocneni-aktualne.mzcr.cz/api/v2/covid-19/kraj-okres-nakazeni-vyleceni-umrti.csv")

df_covid = df_MZ[["okres_lau_kod"]]   # selecting the columns required
df_covid_counts = df_covid.groupby(['okres_lau_kod']).size().reset_index(name='counts')  # group and count okresy - new column was created "counts"
# print (df_covid_counts)
df_merged = df_okresy.set_index("okres_LAU1").join(df_covid_counts.set_index("okres_lau_kod"))
# merge two df based on okres code column
# print (df_merged.head(15))

df_age = df_MZ[["okres_lau_kod","vek"]]
df_age_stats = df_age.groupby(['okres_lau_kod']).agg(["max", "min", "mean", "median"]).reset_index()
df_age_stats.columns = ['okres_lau_kod', 'max_age', "min_age", "avg", "median"]  # rename columns important / generated dont work with Bokeh 

#print (df_age_stats.info())
df_merged2 = pd.merge(df_merged, df_age_stats, left_on='okres_LAU1', right_on='okres_lau_kod')
#print (df_merged2.head(15))
# print (df_merged2.info())
# merge two data frames based on columns with different names 

df_gender = df_MZ[["okres_lau_kod","pohlavi"]]
df_gender_stats_Z = df_gender.groupby('okres_lau_kod')["pohlavi"].apply(lambda x: (x =="Z").sum()).reset_index(name="count")
df_gender_stats_M = df_gender.groupby('okres_lau_kod')["pohlavi"].apply(lambda x: (x =="M").sum()).reset_index(name="count")
df_gender = pd.merge(df_gender_stats_Z, df_gender_stats_M, left_on='okres_lau_kod', right_on='okres_lau_kod')
df_gender.columns = ['okres_lau_kod', 'women', "men"]
df_merged3 = pd.merge(df_merged2, df_gender, left_on='okres_lau_kod', right_on='okres_lau_kod')
#print (df_merged3.head(15))

# be aware - second csv update one week 
df_covid_file2 = df_MZ2[["okres_lau_kod", "kumulativni_pocet_vylecenych", "kumulativni_pocet_umrti"]]
df_h_d = df_covid_file2.groupby(["okres_lau_kod"]).agg(["max"]).reset_index()
df_h_d.columns = ['okres_lau_kod', 'recovered', "dead"]
df_merged4 = pd.merge(df_merged3, df_h_d, left_on='okres_lau_kod', right_on='okres_lau_kod')

# print (df_merged4.head(15))

# ------------------------------------------ last 5 days part ---------------------------------------
d={}
it = 1
for i in range(5): 
 Previous_Date1 = datetime.datetime.today() - datetime.timedelta(days=it)   # previous days (5 days)
 d["time_date{0}".format(it)] = Previous_Date1.strftime("%Y-%m-%d")   
 it += 1                                                                    # dynamic variable dictionary
# print (d["time_date1"])
# print (d)

df_cov = df_MZ[["okres_lau_kod", "datum"]]
df_res1 = df_cov.groupby('okres_lau_kod')["datum"].apply(lambda x: (x ==d["time_date1"]).sum()).reset_index(name="count")
df_res1.columns = ["okres_lau_kod", "1_day_before"] 
# print (df_res1)
df_res2 = df_cov.groupby('okres_lau_kod')["datum"].apply(lambda x: (x ==d["time_date2"]).sum()).reset_index(name="count")
df_res2.columns = ["okres_lau_kod", "2_days_before"] 
# print (df_res2)
df_res3 = df_cov.groupby('okres_lau_kod')["datum"].apply(lambda x: (x ==d["time_date3"]).sum()).reset_index(name="count")
df_res3.columns = ["okres_lau_kod", "3_days_before"] 
# print (df_res3)
df_res4 = df_cov.groupby('okres_lau_kod')["datum"].apply(lambda x: (x ==d["time_date4"]).sum()).reset_index(name="count")
df_res4.columns = ["okres_lau_kod", "4_days_before"] 
# print (df_res4) 
df_res5 = df_cov.groupby('okres_lau_kod')["datum"].apply(lambda x: (x ==d["time_date5"]).sum()).reset_index(name="count")
df_res5.columns = ["okres_lau_kod", "5_days_before"] 
# print (df_res5)

data_frames = [df_res1, df_res2, df_res3, df_res4, df_res5]                # list of all tables for merging
df_merged_try = reduce(lambda  left,right: pd.merge(left,right,on=['okres_lau_kod'],      # merge 5 DF at once
                                            how='outer'), data_frames)
df_merged_try["all_try"] = list(zip(df_merged_try['1_day_before'], df_merged_try['2_days_before'], df_merged_try['3_days_before'],
                                       df_merged_try['4_days_before'],df_merged_try['5_days_before'])) # make zip
# print (df_merged_try)
# -------------------------------------------------- last 5 days part end --------------------------------------

def sizeCircle(df_merged4):                   # make column for point sizes computation (4 categories)
    if df_merged4["counts"] <= 2000:
        val = 10
    elif df_merged4["counts"] <= 4000:          # task: add more categories (e.g. +2) 
        val = 15
    elif df_merged4["counts"] <= 6000:           
        val = 20
    elif df_merged4["counts"] <= 8000:           
        val = 25
    elif df_merged4["counts"] <= 12000:
        val = 30
    else: val = 40 
    return val 

df_merged5 = pd.merge(df_merged4, df_merged_try, left_on='okres_lau_kod', right_on='okres_lau_kod')

df_merged5['size2'] = df_merged4.apply(sizeCircle, axis=1)  # create new column "size 2" with circle sizes              
df_merged5["active_cases"] = df_merged5["counts"] - (df_merged5["recovered"] + df_merged5["dead"])
# recovered and dead are weekly updated 
# create new column "size" - 4 categories of sizes based on "counts" column
# print (df_merged5.head(16))                       
# print (df_merged)
# print (df_merged.info())
 
# -------------------------------------- Interactive map features preparation 

tile = get_provider(Vendors.STAMEN_TERRAIN_RETINA)  # choose map tile provider (we used Stamen - Toner), black and white style background

palette = list(Reds[9])     # color scale selection   --- list(reverse(Reds[9])), list(Reds[9]).reverse()---not work
palette.reverse()     # reverse color order (not from red to white but vice versa)

source = ColumnDataSource(data=df_merged5)  # Tell Bokeh to use df as the source of the data

color_mapper = linear_cmap(field_name = 'counts',      
                           palette = palette, 
                           low = df_merged5['counts'].min(), 
                           high = df_merged5['counts'].max()) # which column will define the colour of the data points
                                                            # Define color mapper 

tooltips = [("Region","@okres"), ("Population","@population"), ("Covid Counts","@counts"), ("Recovered","@recovered"),
            ("Dead","@dead"),("Max age","@max_age"),("Min age","@min_age"), ("Average age","@avg"),
            ("Median age","@median") , ("No. of women","@women"), ("No. of men","@men"), 
            ("Active cases", "@active_cases")]  # set tooltips 
                                                                 # should be shown when the user hovers over a data
hover = """       

    <div style> 
        <div>
            <span style="font-size: 15px;">Region</span>
            <span style="font-size: 17px; font-weight: bold; color: #00E;">@okres</span>           
        </div>
        <div>
            <span style="font-size: 15px;">Total Cases:</span>
            <span style="font-size: 17px; font-weight: bold; color: #F03;">@counts</span>            
        </div>
        <div>
            <span style="font-size: 15px;">Active Cases*:</span>
            <span style="font-size: 17px; font-weight: bold; color: #F0F;">@active_cases</span>            
        </div> 
        <div>
            <span style="font-size: 15px;">Recovered*:</span>
            <span style="font-size: 17px; font-weight: bold; color: #0F0;">@recovered</span>            
        </div>
        <div>
            <span style="font-size: 15px;">Dead*:</span>
            <span style="font-size: 17px; font-weight: bold; color: #000;">@dead</span>            
        </div>
        <div>
            <span style="font-size: 12px;">Age and Gender structure</span>            
        </div>
        <div>
            <span style="font-size: 10px;">Max age:</span>
            <span style="font-size: 12px; font-weight: bold; color: #000;">@max_age</span>            
        </div>
        <div>
            <span style="font-size: 10px;">Min age:</span>
            <span style="font-size: 12px; font-weight: bold; color: #000;">@min_age</span>            
        </div>
        <div>
            <span style="font-size: 10px;">Average age:</span>
            <span style="font-size: 12px; font-weight: bold; color: #000;">@avg</span>            
        </div>       
        <div>
            <span style="font-size: 10px;">No. of women:</span>
            <span style="font-size: 12px; font-weight: bold; color: #000;">@women</span>            
        </div>
        <div>
            <span style="font-size: 10px;">No. of men:</span>
            <span style="font-size: 12px; font-weight: bold; color: #000;">@men</span>            
        </div>             
    </div>
    """    
#<div>
#     <span style="font-size: 10px;">Median age:</span>
#     <span style="font-size: 12px; font-weight: bold; color: #000;">@median</span>            
#</div>


# <div>
#     <span style="font-size: 10px;">Population:</span>
#     <span style="font-size: 12px; font-weight: bold; color: #000;">@population</span>            
#</div>  


#<div>
#    <span style="font-size: 15px;">Last 5 days:</span>
#    <span style="font-size: 17px; font-weight: bold; color: #F03;">@all_try</span>            
# </div> 


#<div>
#    <span style="font-size: 15px;">Location</span>
#    <span style="font-size: 10px; color: #696;">($x, $y)</span> 
#</div>


now = datetime.datetime.now()           # current time
time_date = now.strftime("%Y-%B-%d")     # time format
    
p = figure(title = f'Covid 19 situation in the Czech Republic (absolute values, reported by RHS) {time_date}', x_axis_type="mercator", y_axis_type="mercator",
           x_axis_label = 'Longitude', y_axis_label = 'Latitude', tooltips = hover, tools="tap, pan, box_zoom, reset",
          sizing_mode='stretch_both')  # if you want remove button - tools="pan,wheel_zoom,box_zoom,reset"
# define and create all picture elements
# p.plot_height=700
# p.plot_width=800

p.add_tile(tile)   # add map tiles

ab=p.circle(x = 'mercator_x', y = 'mercator_y', color = color_mapper, source=source, size="size2", 
         fill_alpha = 0.8, line_color = "firebrick", line_dash = [6, 3], line_width = 2)
# to create the data points themselves, add points using mercator coordinates

selected_circle = Circle(fill_alpha=1, fill_color="firebrick", line_color=None)         # select circle
nonselected_circle = Circle(fill_alpha=0.2, fill_color="blue", line_color="firebrick")

ab.selection_glyph = selected_circle                                             # select circle 
ab.nonselection_glyph = nonselected_circle

color_bar = ColorBar(color_mapper=color_mapper['transform'],
            formatter = NumeralTickFormatter(format='0.0[0000]'),
            label_standoff = 13, width=10,location=(0,0))
# define color bar 

p.add_layout(color_bar, 'right')  # location of the color bar 

# ------------------------------------------------------------------------- second tab start ----------------------------
def sizeCircle2(df_merged5):                   # make column for point sizes computation (4 categories)
    if df_merged5["last_5days_sum"] <= 100:
        val = 10
    elif df_merged5["last_5days_sum"] <= 400:          # task: add more categories (e.g. +2) 
        val = 15
    elif df_merged5["last_5days_sum"] <= 700:           
        val = 20
    elif df_merged5["last_5days_sum"] <= 1000:           
        val = 25
    elif df_merged5["last_5days_sum"] <= 1500:
        val = 30
    else: val = 40 
    return val 

tile2 = get_provider(Vendors.CARTODBPOSITRON_RETINA)  # choose map tile provider (we used Stamen - Toner), black and white style background

#palette = Reds[9]     # color scale selection
#palette.reverse()     # reverse color order (not from red to white but vice versa)
df_merged5['last_5days_sum'] = df_merged5['1_day_before'] + df_merged5['2_days_before'] + df_merged5['3_days_before'] + df_merged5['4_days_before'] + df_merged5['5_days_before']
df_merged5['size3'] = df_merged5.apply(sizeCircle2, axis=1)  # create new column "size 2" with circle sizes
df_merged5['rate_per100000'] = round((df_merged5['last_5days_sum'] / df_merged5['population']) * 100000, 2) 

print (df_merged5.head(20))
print (df_merged5['2_days_before'].sum())

source = ColumnDataSource(data=df_merged5)  # Tell Bokeh to use df as the source of the data

color_mapper2 = linear_cmap(field_name = 'last_5days_sum',      
                           palette = palette, 
                           low = df_merged5['last_5days_sum'].min(), 
                           high = df_merged5['last_5days_sum'].max()) # which column will define the colour of the data points
                                                            # Define color mapper 

#tooltips = [("Region","@okres"), ("Population","@population"), ("Covid Counts","@counts"), ("Recovered","@recovered"),
#            ("Dead","@dead"),("Max age","@max_age"),("Min age","@min_age"), ("Average age","@avg"),
#            ("Median age","@median") , ("No. of women","@women"), ("No. of men","@men"), 
#            ("Active cases", "@active_cases")]  # set tooltips 
                                                                 # should be shown when the user hovers over a data
hover2 = """       

    <div style> 
        <div>
            <span style="font-size: 15px;">Region</span>
            <span style="font-size: 17px; font-weight: bold; color: #00E;">@okres</span>           
        </div>
        <div>
            <span style="font-size: 15px;">Previous day:</span>
            <span style="font-size: 17px; font-weight: bold; color: #F03;">@2_days_before</span>           
        </div>
        <div>
            <span style="font-size: 15px;">Last 5 days:</span>
            <span style="font-size: 17px; font-weight: bold; color: #F03;">@all_try</span>            
        </div>
        <div>
            <span style="font-size: 15px;">Last 5 days(sum):</span>
            <span style="font-size: 17px; font-weight: bold; color: #F03;">@last_5days_sum</span>            
        </div>
        <div>
            <span style="font-size: 15px;">Rate per 100k population:</span>
            <span style="font-size: 17px; font-weight: bold; color: #F03;">@rate_per100000</span>            
        </div>       
   </div>
    """    
#<div>
#    <span style="font-size: 15px;">Location</span>
#    <span style="font-size: 10px; color: #696;">($x, $y)</span> 
#</div>


#now2 = datetime.datetime.now()           # current time
#time_date2 = now2.strftime("%Y-%B-%d")     # time format
    
p2 = figure(title = f'Covid 19 situation in the Czech Republic (last 5 days, absolute values, reported by RHS) {time_date}', x_axis_type="mercator", y_axis_type="mercator",
           x_axis_label = 'Longitude', y_axis_label = 'Latitude', tooltips = hover2, tools="tap, pan, box_zoom, reset",
           sizing_mode='stretch_both')  # if you want remove button - tools="pan,wheel_zoom,box_zoom,reset"
# define and create all picture elements
#p2.plot_height=700
#p2.plot_width=800

p2.add_tile(tile2)   # add map tiles

ab2=p2.circle(x = 'mercator_x', y = 'mercator_y', color = color_mapper2, source=source, size="size3", 
         fill_alpha = 0.8, line_color = "firebrick", line_dash = [6, 3], line_width = 2)
# to create the data points themselves, add points using mercator coordinates

# selected_circle = Circle(fill_alpha=1, fill_color="firebrick", line_color=None)         # select circle
# nonselected_circle = Circle(fill_alpha=0.2, fill_color="blue", line_color="firebrick")

ab2.selection_glyph = selected_circle                                             # select circle 
ab2.nonselection_glyph = nonselected_circle

color_bar2 = ColorBar(color_mapper=color_mapper2['transform'],
            formatter = NumeralTickFormatter(format='0.0[0000]'),
            label_standoff = 13, width=10,location=(0,0))
# define color bar 

p2.add_layout(color_bar2, 'right')  # location of the color bar

# -------------------------------------------------------------------------------- second tab end -------------------

# -------------------------------------------------------------------------------- third tab start -----------------
color_mapper3 = linear_cmap(field_name = 'rate_per100000',      
                           palette = palette, 
                           low = df_merged5['rate_per100000'].min(), 
                           high = df_merged5['rate_per100000'].max())

p4 = figure(title = "Population/Covid counts", tooltips="@okres: (population: @population, Last 5 days: @last_5days_sum)",
           tools="tap, pan, box_zoom, reset")
p4.plot_height=700
p4.plot_width=800
p4.circle('population','rate_per100000', color = color_mapper3, source=df_merged5, 
          fill_alpha=0.8, size=12, line_color = "green")   #fill_color="red", legend="okres"
p4.below[0].formatter.use_scientific = False           # disable 4.00e+5 numbers for population (below=x ax, left=y ax)
p4.xaxis.axis_label = 'Population'
p4.yaxis.axis_label = 'Covid rate per 100k(last 5 days)'
#p4.grid.bounds = (2, 4)
p4.xaxis.major_tick_line_color = "firebrick"
p4.xaxis.major_tick_line_width = 3
p4.xaxis.minor_tick_line_color = "orange"
# p4.legend.location = "top_left"

# -------------------------------------------------------------------------------- third tab end ------------------
# -------------------------------------------------------------------------------- text tab start -----------------
p6 = Paragraph(text="""DISCLAIMER OF LIABILITY AND ACCURACY OF DATA.  
     The information contained in this website has been supplied to IT4Innovations from a variety of sources, 
     and is subject to change at any time without notice.
     IT4Innovations gives no assurance or warranty that information on this site is current and accurate, and take no 
     responsibility for matters arising from changed circumstances or other information or material which may affect the 
     currency or accuracy of information on this site.
     The User assumes all responsibility and risk for the use of this site.
     """)
# width=800, height=700)
p7 = Paragraph(text="""WARNING: The Covid-19 maps contain cases reported by the 
     regional hygiene stations only. Data updates are published at different delay levels.
     This may lead to discrepancies between the map numbers and the numbers published by other sources.
     Users are advised to use all data with caution and awareness of their limitations.
     """)

data = dict(
        item=["Description","Spatial unit", "Version", "Update","Status", "Type", "Background maps", "Coordinate system",
             "Programming language", "Data source", "Records with *", "Organization","Contact"],
        des=["Geographical distribution of Covid-19 cases in the Czech Republic","Regions", 1.01, "3-times per day",
            "under development", "Interactive map", "OpenStreetMap", "Universal Transverse Mercator (UTM)", "Python 3.x",
            "The Ministry of Health (open dataset)", "data are delayed (usually few days)", 
            "IT4Innovations, ADAS lab", "michal.podhoranyi@vsb.cz" ],
    )
source = ColumnDataSource(data)

columns = [
        TableColumn(field="item", title=" "),
        TableColumn(field="des", title=" "),
    ]
p5 = DataTable(source=source, columns=columns, width=800, height=700, index_position=None)



# -------------------------------------------------------------------------------- text tab end -------------------
df_MZ3 = pd.read_csv ("https://onemocneni-aktualne.mzcr.cz/api/v2/covid-19/nakaza.csv")
print(df_MZ3)

title_name = 'Total cases / Last five days (abs.)' 
price = df_MZ3["kumulativni_pocet_nakazenych"].max()
percentage = df_MZ3["prirustkovy_pocet_nakazenych"].iloc[-1]
percentage2 = df_MZ3["prirustkovy_pocet_nakazenych"].iloc[-2]
percentage3 = df_MZ3["prirustkovy_pocet_nakazenych"].iloc[-3]
percentage4 = df_MZ3["prirustkovy_pocet_nakazenych"].iloc[-4]
percentage5 = df_MZ3["prirustkovy_pocet_nakazenych"].iloc[-5]
#template for the text and numbers that appear
template=("""
      <div class='content'> 
       <div class='name'> {title_name} </div>
        <span class='number' style='color: {colour2};'>{price}</span>         
        <span class='percentage' style='color: {colour};'> /<small>+</small>{percentage}</span>
        <span class='percentage' style='color: {colour};'> <small>+</small><small>{percentage2}</small></span>
        <span class='percentage' style='color: {colour};'> <small>+</small><small><small>{percentage3}<small></small></span>
        <span class='percentage' style='color: {colour};'> <small>+</small><small><small>{percentage4}<small></small></span>
        <span class='percentage' style='color: {colour};'> <small>+</small><small>{percentage5}</small></span>
      </div>
      """)
# <small>{price_unit}</small> </span>
# <small>%</small>
text = template.format(title_name = title_name,
                   price=price,
                   colour2='#FF0000',
                   # price_unit='k',
                   percentage=percentage,
                   percentage2=percentage2,
                   percentage3=percentage3,
                   percentage4=percentage4,
                   percentage5=percentage5,
                   colour='#97D389')

div = Div(text=text, style={'font-size': '200%', 'color': 'black'})


# --------------------------------------------------------------------------------------------- counter total cases div end-
l1 = column(div,p4)
l2 = column (p5, p6, p7)

tab1 = Panel(child=p, title="Covid19 Total")    # define new tab (p is figure)
tab2 = Panel (child=p2, title="Last 5 days") 
tab3 = Panel (child=l1, title="Rate per 100k")
tab4 = Panel (child=l2, title="Info")
tabs = Tabs(tabs=[tab1, tab2, tab3, tab4])        

output_notebook()  # display in notebook

#output_file('C:/Users/pod172/Desktop/data_cov19_analysis/output_bokeh_html/Bokeh_ver4.html', 
#            title='Covid 19 version 4')   # export as HTML
output_file('Bokeh_ver1.html',      # '../Desktop/Bokeh_ver5.html' 
            title='Covid 19 version 1')

show(tabs)

               okres kraj_NUTS3   latitude  longitude  population  \
0            Bruntál      CZ080  49.988232  17.463576       91597   
1      Frýdek-Místek      CZ080  49.682561  18.358651      214660   
2            Karviná      CZ080  49.861098  18.543283      246324   
3         Nový Jičín      CZ080  49.596599  18.015774      151577   
4              Opava      CZ080  49.940066  17.894213      176236   
5      Ostrava-Město      CZ080  49.820923  18.262524      320145   
6            Jeseník      CZ071  50.223847  17.200008       37968   
7            Olomouc      CZ071  49.590463  17.254502      235472   
8          Prostějov      CZ071  49.472283  17.111340      108646   
9             Přerov      CZ071  49.451107  17.447462      129512   
10           Šumperk      CZ071  49.964742  16.974617      120417   
11          Kroměříž      CZ072  49.294681  17.393367      105343   
12  Uherské Hradiště      CZ072  49.067078  17.481180      142226   
13            Vsetín      CZ072  4

Loading BokehJS ...